# Ranking et search engine

C'est un petit exemple de ranking avec un très petit jeu de données, trop petit pour que le modèle soit performant, mais le code peut être réutilisé pour des exemples de taille raisonnable. C'est à dire probablement pas pour apprnedre un moteur de recherche.

In [1]:
%matplotlib inline

In [2]:
from papierstat.datasets import load_search_engine_dataset
X, y, qid = load_search_engine_dataset()
X[:5,:6].todense()

matrix([[3., 3., 0., 0., 3., 1.],
        [3., 0., 3., 0., 3., 1.],
        [3., 0., 2., 0., 3., 1.],
        [3., 0., 3., 0., 3., 1.],
        [3., 0., 3., 0., 3., 1.]])

In [3]:
X.shape

(582, 136)

Le tableau ``qid`` contient l'identifiant de la requête, toutes les lignes associées à un identifiant correspondent à des résultats associés à cette requête. Dans ce jeu, il y a 7 requêtes distinctes.

In [4]:
set(qid)

{1, 16, 31, 46, 61, 76, 91}

On peut essayer d'abord [XGBoost](http://xgboost.readthedocs.io/en/latest/python/index.html).

In [5]:
X_train, y_train, qid_train = load_search_engine_dataset()

In [6]:
import pandas
df = pandas.DataFrame(qid_train)
df['c'] = 1
df.columns = ['qid', 'c']
gr = df.groupby('qid').count()
gr

,c
qid,
1,86
16,106
31,92
46,120
61,59
76,45
91,74


In [7]:
from xgboost import DMatrix
dtrain = DMatrix(data=X_train, label=y_train)
dtrain.set_group(gr.as_matrix())

In [8]:
from xgboost import XGBRegressor, train
rk = train(params={'objective': 'rank:ndcg'}, dtrain=dtrain, num_boost_round=10)

In [9]:
X_test, y_test, qid_test = load_search_engine_dataset(False)

In [10]:
import pandas
df = pandas.DataFrame(qid_test)
df['c'] = 1
df.columns = ['qid', 'c']
gr = df.groupby('qid').count()
dtest = DMatrix(data=X_test)
dtest.set_group(gr.as_matrix())

In [11]:
pred = rk.predict(dtest)

On peut calculer l'erreur au carré.

In [12]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, pred)

0.816864587903257

Mais cela n'est valeur que si le score a un sens, ce qui est le cas ici. Si ce n'est pas le cas, il est possible d'évaluer les résultats avec la corrélation des rangs des résultats ([coefficient de Kendall](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient)).